In [1]:
import tweepy 
import json
import datetime
import sqlite3
from sqlite3 import Error

In [2]:
def read_outh_file(outh_filename):
    keys = []
    with open(outh_filename) as fp:
        line = fp.readline()
        while line:
            keys.append(line.strip('\n'))
            line = fp.readline()

    consumer_key = keys[0]
    consumer_secret_key = keys[1]
    access_token = keys[2]
    access_secret_token = keys[3]
    
    return consumer_key,consumer_secret_key,access_token,access_secret_token

In [3]:
def autorize_twitter_api():

    consumer_key,consumer_secret_key,access_token,access_secret_token = read_outh_file("outh.txt")
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret_key)
    auth.set_access_token(access_token, access_secret_token)
    
    return auth

In [4]:
def create_connection(db_file):
    connection = None
    try:
        connection = sqlite3.connect(db_file)
        return connection
    except Error as e:
        print(e)
 
    return connection

In [5]:
def create_all_tables():
    
    sql_create_sentiments_table = """ CREATE TABLE IF NOT EXISTS sentiment_table (
                                    sentiment_id text PRIMARY KEY,
                                    user_name text NOT NULL,
                                    tweet text,
                                    longitude float,
                                    latitude float,
                                    sentiment text
                                ); """
    create_table(sql_create_sentiments_table)

In [6]:
def create_table(create_table_sql):
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
        conn.commit()
        c.close()   
    except Error as e:
        print(e)

In [7]:
def insert_into_table(insert_sql, parameters):
    try:
        c = conn.cursor()
        c.execute(insert_sql, parameters)
        conn.commit()
        c.close()   
    except Error as e:
        print(e)

In [8]:
def insert_into_sentiment_table(data):
        
    if data['coordinates'] is not None:
        sentiment_id = term_to_search + "_"+ str(datetime.datetime.now().timestamp())
        user_name = data['user']['name']
        tweet = data['text']
        longitude = data['coordinates'][0]
        latitude = data['coordinates'][1]
        sentiment = ""
    elif data['place'] is not None:
        bbox = data['place']['bounding_box']['coordinates']
        sentiment_id = term_to_search + "_"+ str(datetime.datetime.now().timestamp())
        user_name = data['user']['name']
        tweet = data['text']
        longitude = bbox[0][0][0]
        latitude = bbox[0][0][1]
        sentiment = ""
    else:
        sentiment_id = None
    
    if sentiment_id is not None:
        
        insert_sql = '''INSERT INTO sentiment_table VALUES (?, ?, ?, ?, ?, ?)'''
        parameters = (sentiment_id, user_name, tweet, longitude, latitude, sentiment)
        
        print(parameters)
        
        insert_into_table(insert_sql, parameters)   
        
    return

In [9]:
class streamListener(tweepy.StreamListener):

    
    def on_data(self, raw_data):
       
        
        try:
                       
            data = json.loads(raw_data) 
            
            insert_into_sentiment_table(data)
                        
            
        except Exception as e:
            print(e)
    
    def on_error(self, status_code):
        if status_code == 420:
            return False

In [ ]:
if __name__ == "__main__": 

    global term_to_search, conn
    
    database = "Twitter_Database.db"   
    conn = create_connection(database)
    
    create_all_tables()          
            
    term_to_search = "politics"
    
    api = tweepy.API(wait_on_rate_limit_notify=True)

    streamer = tweepy.Stream(auth=autorize_twitter_api(), listener=streamListener(api))
    streamer.filter(languages=["en"], track=[term_to_search])

('politics_1573877038.945928', '🇬🇧😂🏴God®🇲🇳Khuda™️😂Bhagwan©️😡👍☀️🇧🇹😒🗺📌', 'And from the starting  We used to say  you as  (  RAM  )  Sena in whatever field you are like in farming or agricul… https://t.co/MrLGvRHlmu', 78.861222, 20.755232, '')
('politics_1573877054.104458', 'Cam_Walker', "'Shabby' and 'mediocre' kind of sums up his government, not just these policies.", 144.167985, -37.106867, '')
('politics_1573877078.923509', 'Kris Colvin', 'It’s tough to be a criminal at the highest levels of governance. Not sorry it’s not easier on this fecal matter who… https://t.co/JEGQxozLkX', -82.39106, 34.844855, '')
('politics_1573877088.398602', 'Maribel Brandt', 'Trump ignores Pentagon advice and intervenes in military war crimes cases\n\nhttps://t.co/bRLAgZEgTe', -96.706144, 32.829815, '')
('politics_1573877120.811808', 'Hitesh@patel', "This is how  opposition work don't Dream for Chair dream about India U will be learn something  between Politics &amp;… https://t.co/rfrG6ZoZlA", 72.430081, 2

('politics_1573878514.627441', 'Jeremy C. Owens', 'TV cut to Brad Stevens just sitting on the bench around some other dudes in suits at the Warriors game, and I thoug… https://t.co/IUHmZJa7e3', -122.332411, 37.720367, '')
('politics_1573878564.870308', 'tsunamiazul', "No you don't know these guys  huh ?  You're a filthy criminal.  #TrumpCrimeFamilyForPrison https://t.co/wEsj4Quq3S", -104.858332, 39.565982, '')
('politics_1573878575.996172', 'Alfredo Velázquez', 'Now I understand why #Trump was unrespectful and humilliated @JustinTrudeau . #Trudeau is a looser.', -99.259495, 19.386371, '')
('politics_1573878631.370739', 'Gabe Klein', 'Nothing to see here', -118.668404, 33.704538, '')
('politics_1573878727.670611', 'Magnificent bastard. Long may you rock..', 'Sssshhhhh, less politics... more baby Yoda speak.', -110.005527, 53.248878, '')
('politics_1573878801.853634', 'Dan Rodimer for Congress', 'I had a wonderful time at #ShiratShalomCongregation tonight with @mikemears and @NVGOP. Grea

('politics_1573880148.827081', 'Cyndy🗽Safety-Pin', "State Dept Aide David Holmes' Opening Statement.\nHe flew from Ukraine for today's closed door hearing after reading… https://t.co/AiqHSuqi46", -76.22808, 36.672684, '')
('politics_1573880198.161983', 'Girl Friday... but', 'Tonight’s lesson is: don’t talk politics with your fling - cause neither of you is getting laid and now you don’t have a fling 😬🤦🏻\u200d♀️', -64.237659, 43.366298, '')
database is locked
